In [10]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import feature_process_helper
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import pickle
%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')
del y_train['id']
X_train, X_test = feature_process_helper.dates(X_train, X_test)
X_train, X_test = feature_process_helper.construction(X_train, X_test)
X_train, X_test = feature_process_helper.bools(X_train, X_test)
X_train, X_test = feature_process_helper.locs(X_train, X_test)
X_train, X_test = feature_process_helper.removal(X_train, X_test)
X_train, X_test = feature_process_helper.dummies(X_train, X_test)

In [3]:
print np.shape(X_train)
print np.shape(X_test)

(59400, 3754)
(14850, 3754)


### Logistic Regression classifier

In [18]:
features = pickle.load(open(os.path.join('pickles', 'rf3_features.pkl'), 'rb'))

In [21]:
lg = LogisticRegression(random_state=0)

scores = cross_val_score(lg, X_train[list(features['variable'][:300])], y_train.values.ravel(), scoring='accuracy', cv=3)
scores.mean()

0.58141414141414149

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_range = [0.01, 0.1, 1.0, 10.0, 100.0]

param_grid = [{'C': param_range}]

gs = GridSearchCV(estimator=LogisticRegression(random_state=0),
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=2,
                  n_jobs=-1)

scores = cross_val_score(gs, X_train[list(features['variable'][:300])], y_train.values.ravel(), scoring='accuracy', cv=3)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy: 0.582 +/- 0.009


### Generate submission file

In [161]:
predictions = rf.predict(X_test)
y_test = pd.read_csv('y_test.csv')
pred = pd.DataFrame(predictions, columns = [y_test.columns[1]])
del y_test['status_group']
y_test = pd.concat((y_test, pred), axis = 1)
y_test.to_csv('y_test.csv', sep=",", index = False)